# Kurs 3

## Exkurs: Map

Die Funktion ```map``` funktioniert ähnlich wie broadcasting.

In [ ]:
# Wie gewohnt
map(exp, [1, 2, 3])

In [ ]:
exp.([1, 2, 3])

Es gibt allerdings ein paar kleine Unterschiede:

In [ ]:
[1, 2, 3, 4] .+ [1 2 3 4]

In [ ]:
map(+, [1, 2, 3, 4], [1 2 3 4])

Das liegt daran, dass ```map``` die beiden Inputs nach Reißverschlussprinzip miteinander kombiniert, also etwa so:

In [ ]:
zip([1, 2, 3, 4], [1 2 3 4])

Dementsprechend schert sich ```map``` in unserem Beispiel nicht um Dimensionen (Zeilen vs. Spalten) und hat auch kein Problem damit, wenn die Länge nicht übereinstimmt:

In [ ]:
map(+, [1, 2, 3], [1 2 3 4])

Tatsächlich ist map ein super allgemeines Konzept, das zum Beispiel auch beim highperformance computing auftaucht. Dort hat man nämlich üblicherweise sogenannte map-reduce patterns. Die Intuition hierbei ist: Man hat in der Praxis immer hochparallele Systeme (viele Rechenkerne oder GPU(s), TPU(s)) und muss daher eine Aufgabe in möglichst gleiche Teile zerlegen, die individuell ausgerechnet (map) und nachher wieder zusammengeführt (reduce) werden[^1]. Ein Beispiel wäre etwa das maximale Element einer großen Matrix ```A``` zu finden. Dabei hätten wir vielleicht verschiedene Maschinen, die dann jeweils das Maximum einer Spalte ausrechnen und an unseren Hauptrechner (host) zurückgeben. Dieser braucht dann nur noch die Ergebnisse zu reducen, dann sind wir fertig:

[^1]: 
    Aus mathematischer Sicht braucht man für map-reduce Assoziativität (klar für das Maximum) und Existenz eines neutralen Elements (das wäre dafür gerade ```-Inf```). Mehr Infos dazu [hier](https://en.wikipedia.org/wiki/MapReduce#Theoretical_background).

In [ ]:
mat = [1 2; 3 4] # Beispielmatrix
results = map(x -> maximum(x), eachcol(mat)) # könnte beispielsweise auf verschiedenen Rechenkernen stattfinden
println(results)
reduce(max, results) # wir brauchen hier max, weil das ein binärer Operator ist

Ein letzter Tipp: Wenn die „gemapte“ Funktion keinen Output hat, verwendet man ```foreach```.

## Rekursion

Mit Rekursion meinen wir den Prozess, dass Funktionen sich *selbst* wieder aufrufen können.

In [ ]:
function pow(a, n) # a^n
    if n == 0 # wir wollen irgendwann in diesem case landen
        return 1
    elseif n < 0
        return(pow(a, n + 1) / a) # a^n = a^(n+1) / a
    else
        return(pow(a, n - 1) * a) # a^n = a^(n-1) * a
    end
end

pow(2, 4)

## ```for```-Loops

Untenstehend ist ein ziemlich unpraktischer Weg, um sich die Zahlen 1-10 auszugeben. 

In [ ]:
println(1)
println(2)
println(3)
println(4)
println(5)
println(6)
println(7)
println(8)
println(9)

Wir wollen nämlich repetitive Aufgaben durch sogenannte Loops (Schleifen) lösen.

In [ ]:
function print_numbers(first_number, last_number) 
    for i in first_number:last_number
        println(i)
    end
end

print_numbers(1, 9)

Es kann nicht nur über Ranges von Zahlen (wie ```first_number:last_number```) iteriert werden, sondern auch über beliebige Elemente in einem Array:

In [ ]:
array1 = ["H", "A", "L", "L", "O", 1, 2, 3]

for element in array1
    print(element) # Ausgabe ohne neue Zeile am Ende
end

Auch bei der Erstellung von Arrays kann man Loops verwenden:

In [ ]:
array2 = [2 * i^2 for i in 1:10]

Wenn wir dagegen umgekehrt über die Indizes eines Arrays loopen wollen, dann geht das zwar durchaus mit ```1:length(array2)```, man sollte tendenziell aber eher den Befehl ```eachindex``` nutzen:

In [ ]:
for i in eachindex(array2)
    println(i)
end

Ähnlich funktioniert ```axes``` für höherdimensionale Objekte:

In [ ]:
for j in axes(mat, 2) # loope über Spalten
    println(mat[1, j]) # fixiere außerdem erste Zeile
end

## ```while```-Loops

Hier können wir nun solange (```while```) einen Befehl ausführen, wie eine bestimmte Bedingung erfüllt ist:

In [ ]:
function countdown_from_n(n) 
    while n > 0
        println(n)
        n -= 1
    end
end

countdown_from_n(5)

Potenziell kann `while` ziemlich bösartig sein, weil etwa `while true ... end` nie terminiert (Endlosschleife) und daher unser Programm hängen bleibt oder abstürzt.

Eine kleine Randbemerkung: Anders als in vielen anderen Sprachen (insbesondere Interpretersprachen wie Python oder R) müssen wir uns in Julia meist nicht davor fürchten, dass selbstgeschriebene Loops inperformant werden. Denn an irgendeiner Stelle muss ja der Loop implementiert sein (außer man schreibt in Assembler), in Python etwa meistens in C/C++. Das bedeutet: Schneller, spezialisierter Code aus C/C++ (etwa NumPy) wird für, sagen wir, Matrizenrechnung eingebunden und ersetzt Python-Loops. In Julia entfällt dieser Schritt, weil sowieso das meiste direkt (nativ) in Julia geschrieben wird bzw. weil Julia schon sehr performant ist. Eine Ausnahme wären etwa hochoptimierte Bibliotheken für Lineare Algebra (aber die werden sowieso in jeder Sprache extern eingebunden).

Zusammengefasst bietet uns sogenannte Vektorisierung wie in R oder Matlab per se keine Vorteile (das wäre gerade sowas wie broadcasting oder map in Julia). Nichtsdestotrotz steigt der Rechenaufwand verschachtelter Loops natürlich exponentiell und es gibt einige Tricks, die man zum Beschleunigen nutzen kann – dazu später mehr.